In [2]:
import pandas as pd
import pandera as pa
import datetime as dt

In [3]:
colunas = ['codigo_ocorrencia','codigo_ocorrencia2','ocorrencia_classificacao','ocorrencia_cidade','ocorrencia_uf'
          ,'ocorrencia_aerodromo','ocorrencia_dia','ocorrencia_hora','total_recomendacoes']

# o parâmetro dayfirst formata a data para o dia vir primeiro
df = pd.read_csv('ocorrencia.csv', sep=';',parse_dates=['ocorrencia_dia'], dayfirst=True)
df = df[colunas]
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,***,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,***,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,***,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,####,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,****,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [4]:
# Limpeza de dados

# ocorrencia_uf limpar -> **
# ocorrencia_aerodromo Limpar -> ###! -> ### -> **** -> *****
# ocorrencia_hora Limpar -> null

df.loc[df.ocorrencia_uf == '**', ['ocorrencia_uf']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '###!',['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '###',['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '####',['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '***',['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '****',['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '*****',['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_hora == 'NULL',['ocorrencia_hora']] = pd.NA

# Toda limpeza poderia ser feita de forma mais simples em todo df, como segue abaixo

#df.replace(['**','***','****','*****','###!','###','####','NULL'],pd.NA, inplace=True)
#df

In [5]:
# Soma ocorrências de valores NA por coluna
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        1905
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [6]:
# substitui valores NA por 0
df.fillna(0, inplace=True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,0,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,0,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,0,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,0,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [7]:
# Esses 3 casos devem ser tratados, caso contrário não passa na validação.
df.loc[1099, ['ocorrencia_uf']] = pd.NA # valor anterior 0
df.loc[4728, ['ocorrencia_uf']] = pd.NA # valor anterior 0
df.loc[2974, ['ocorrencia_hora']] = '00:00:00'  # valor anterior 0

In [8]:
# Etapa de validação dos dados
# Schema pandera para validar colunas e Tipo de dados de um DataFrame
schema = pa.DataFrameSchema(
      
      columns = {'codigo_ocorrencia':pa.Column(int),
                 'codigo_ocorrencia2':pa.Column(int),
                 'ocorrencia_classificacao':pa.Column(str),
                 'ocorrencia_cidade':pa.Column(str),
                 'ocorrencia_uf':pa.Column(str,pa.Check.str_length(2,2),nullable=True),
                 'ocorrencia_aerodromo':pa.Column(str),
                 'ocorrencia_dia':pa.Column(dt.datetime),
                 'ocorrencia_hora':pa.Column(str,pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'),nullable=True), # permitir valores nulos e valida horas via REgex
                 'total_recomendacoes':pa.Column(int)}
)

schema.validate(df)
## Mostrando tipos pós validação
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [9]:
# Um pouco de manipulação
# Colocando o codigo_ocorrencia como index
df.set_index('codigo_ocorrencia',inplace=True)
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0
45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,0,2012-01-06,17:00:00,0
45407,45407,ACIDENTE,SÃO SEPÉ,RS,0,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...
80458,80458,ACIDENTE,JATAÍ,GO,0,2021-12-30,20:30:00,0
80452,80452,ACIDENTE,MARACAÍ,SP,0,2021-12-31,09:30:00,0
80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0


In [10]:
# Voltando ao index original
df.reset_index(inplace=True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,0,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,0,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,0,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,0,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [11]:
# Varre o df e na linha onde ocorrencia_uf for igual a SP ocorrencia_classificacao recebe "GRAVE" 
df.loc[df.ocorrencia_uf == 'SP', ['ocorrencia_classificacao']] = 'GRAVE'
df.loc[df.ocorrencia_uf == 'SP']

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,45331,45331,GRAVE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
3,45401,45401,GRAVE,SÃO SEBASTIÃO,SP,0,2012-01-06,17:00:00,0
5,52243,52243,GRAVE,UBATUBA,SP,0,2012-01-06,14:30:00,0
6,50713,50713,GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
13,45408,45408,GRAVE,ELDORADO,SP,0,2012-01-11,13:45:00,1
...,...,...,...,...,...,...,...,...,...
5148,80441,80441,GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-12-20,18:15:00,0
5157,80453,80453,GRAVE,CAMPINAS,SP,SBKP,2021-12-29,09:00:00,0
5158,80454,80454,GRAVE,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0
5161,80456,80456,GRAVE,SÃO PAULO,SP,SBSP,2021-12-30,13:15:00,0


### para preencher os valores NA apenas em uma coluna podemos fazer como segue abaixo

df.fillna(value={'ocorrencia_aerodromo': 0}, inplace=True)
df

In [12]:
# Criando uma coluna de backup e excluindo a coluna original

df['classificacao'] = df['ocorrencia_classificacao']
df.drop(['ocorrencia_classificacao'],axis=1, inplace=True)
df

# df.dropna(inplace = True) exclui todas as linhas que tenham algum valor NA
# df.dropna(subset=['classificacao'],inplace = True) exclui todas as linhas que tenham algum valor NA olhando para coluna classificacao

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
0,52242,52242,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,INCIDENTE
1,45331,45331,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,GRAVE
2,45333,45333,VIAMÃO,RS,0,2012-01-06,13:00:00,0,ACIDENTE
3,45401,45401,SÃO SEBASTIÃO,SP,0,2012-01-06,17:00:00,0,GRAVE
4,45407,45407,SÃO SEPÉ,RS,0,2012-01-06,16:30:00,0,ACIDENTE
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,JATAÍ,GO,0,2021-12-30,20:30:00,0,ACIDENTE
5163,80452,80452,MARACAÍ,SP,0,2021-12-31,09:30:00,0,GRAVE
5164,80457,80457,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0,INCIDENTE GRAVE
5165,80460,80460,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0,INCIDENTE


In [13]:
# Exclui linhas duplicadas
df.drop_duplicates(inplace=True)

# Verifica se a linha ocorrencia_uf tem valores nulos
df.loc[df.ocorrencia_uf.isnull()]
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
0,52242,52242,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,INCIDENTE
1,45331,45331,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,GRAVE
2,45333,45333,VIAMÃO,RS,0,2012-01-06,13:00:00,0,ACIDENTE
3,45401,45401,SÃO SEBASTIÃO,SP,0,2012-01-06,17:00:00,0,GRAVE
4,45407,45407,SÃO SEPÉ,RS,0,2012-01-06,16:30:00,0,ACIDENTE
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,JATAÍ,GO,0,2021-12-30,20:30:00,0,ACIDENTE
5163,80452,80452,MARACAÍ,SP,0,2021-12-31,09:30:00,0,GRAVE
5164,80457,80457,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0,INCIDENTE GRAVE
5165,80460,80460,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0,INCIDENTE


In [14]:
# mostra cidades com mais de 10 recomendações
df.loc[df.total_recomendacoes>10, ['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
844,BRASÍLIA,11
1669,SANTOS,13
2804,VITÓRIA,12


In [15]:
# Filtra estados e cidades onde ocorreram incidentes graves
df_incidentes_graves = df.loc[df.classificacao == 'INCIDENTE GRAVE', ['ocorrencia_cidade','ocorrencia_uf']]

# Ordena os estados por qtd de ocorrências de incidentes graves
df_incidentes_graves = df_incidentes_graves['ocorrencia_uf'].value_counts()
df_incidentes_graves

MG    67
GO    66
PR    50
MT    47
RS    43
RJ    40
PA    38
BA    32
MS    26
AM    22
SC    14
CE    14
MA    12
DF    10
ES    10
TO    10
AC     9
RO     9
PE     9
PI     7
RR     7
AL     5
PB     5
RN     4
SE     2
AP     1
Name: ocorrencia_uf, dtype: int64

In [54]:
# Filtrar estados por qtd de incidentes graves ou graves durante o ano de 2021
df_datas = df.set_index('ocorrencia_dia')
df_datas = df_datas.loc['2021-01-01':'2021-12-31',['ocorrencia_uf','classificacao']]
df_datas = df_datas.loc[((df_datas.classificacao == 'GRAVE') | (df_datas.classificacao == 'INCIDENTE GRAVE'))]
df_datas.value_counts()

ocorrencia_uf  classificacao  
SP             GRAVE              131
GO             INCIDENTE GRAVE     11
MS             INCIDENTE GRAVE      8
MT             INCIDENTE GRAVE      8
PR             INCIDENTE GRAVE      6
RS             INCIDENTE GRAVE      5
MG             INCIDENTE GRAVE      5
BA             INCIDENTE GRAVE      4
PA             INCIDENTE GRAVE      3
RJ             INCIDENTE GRAVE      3
SC             INCIDENTE GRAVE      3
RR             INCIDENTE GRAVE      3
AC             INCIDENTE GRAVE      2
CE             INCIDENTE GRAVE      2
DF             INCIDENTE GRAVE      2
RO             INCIDENTE GRAVE      2
PB             INCIDENTE GRAVE      1
RN             INCIDENTE GRAVE      1
ES             INCIDENTE GRAVE      1
TO             INCIDENTE GRAVE      1
dtype: int64

In [17]:
# filtra incidentes graves e graves pelo aerodromo SBGR
filtro01 = ((df.classificacao == 'GRAVE') | (df.classificacao == 'INCIDENTE GRAVE'))
filtro02 = df.ocorrencia_aerodromo == 'SBGR'
df.loc[filtro01 & filtro02]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
1,45331,45331,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,GRAVE
86,45653,45653,GUARULHOS,SP,SBGR,2012-02-24,20:04:00,0,GRAVE
106,45601,45601,GUARULHOS,SP,SBGR,2012-03-07,17:40:00,0,GRAVE
187,46048,46048,GUARULHOS,SP,SBGR,2012-04-24,16:40:00,1,GRAVE
193,45801,45801,GUARULHOS,SP,SBGR,2012-04-27,03:54:00,0,GRAVE
...,...,...,...,...,...,...,...,...,...
4922,80170,80170,GUARULHOS,SP,SBGR,2021-07-07,15:36:00,0,GRAVE
5014,80276,80276,GUARULHOS,SP,SBGR,2021-09-08,12:05:00,0,GRAVE
5015,80437,80437,GUARULHOS,SP,SBGR,2021-09-08,12:05:00,0,GRAVE
5055,80332,80332,GUARULHOS,SP,SBGR,2021-10-15,13:42:00,0,GRAVE


In [18]:
# filtra ocorrências de cidades que comecem com a letra C
filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
6,50713,50713,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0,GRAVE
8,45391,45391,CONCEIÇÃO DAS ALAGOAS,MG,0,2012-01-08,16:00:00,0,ACIDENTE
15,45409,45409,CÁCERES,MT,0,2012-01-14,10:00:00,0,ACIDENTE
18,50714,50714,CAMOCIM,CE,SNWC,2012-01-17,18:25:00,0,INCIDENTE GRAVE
22,45390,45390,CACHOEIRA DOURADA,GO,0,2012-01-20,21:00:00,0,ACIDENTE
...,...,...,...,...,...,...,...,...,...
5142,80434,80434,CARACOL,MS,0,2021-12-14,21:35:00,0,ACIDENTE
5144,80447,80447,CONFINS,MG,SBCF,2021-12-15,17:10:00,0,INCIDENTE
5150,80445,80445,CHUPINGUAIA,RO,0,2021-12-20,12:05:00,0,ACIDENTE
5157,80453,80453,CAMPINAS,SP,SBKP,2021-12-29,09:00:00,0,GRAVE


In [19]:
# filtra ocorrências de cidades que terminam com a letra A
filtro = df.ocorrencia_cidade.str[-1] == 'A'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
5,52243,52243,UBATUBA,SP,0,2012-01-06,14:30:00,0,GRAVE
9,52244,52244,UBERLÂNDIA,MG,SBUL,2012-01-08,22:13:00,0,INCIDENTE
17,52246,52246,BRASÍLIA,DF,SBBR,2012-01-16,16:47:00,0,INCIDENTE
21,45392,45392,BRASÍLIA,DF,0,2012-01-19,21:30:00,2,ACIDENTE
22,45390,45390,CACHOEIRA DOURADA,GO,0,2012-01-20,21:00:00,0,ACIDENTE
...,...,...,...,...,...,...,...,...,...
5152,80465,80465,VITÓRIA,ES,SBVT,2021-12-21,22:25:00,0,INCIDENTE
5158,80454,80454,ARAÇATUBA,SP,SBAU,2021-12-29,21:35:00,0,GRAVE
5159,80455,80455,SANTA RITA,PB,0,2021-12-29,18:50:00,0,INCIDENTE GRAVE
5165,80460,80460,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0,INCIDENTE


In [20]:
# filtra ocorrências de cidades que terminam com os caracters MA
filtro = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
249,45939,45939,UMUARAMA,PR,0,2012-05-30,19:00:00,0,ACIDENTE
273,46180,46180,AURIFLAMA,SP,0,2012-06-16,19:30:00,0,GRAVE
971,49113,49113,NOVA LIMA,MG,0,2013-06-15,12:00:00,4,INCIDENTE GRAVE
974,48799,48799,CANUTAMA,AM,0,2013-06-18,21:30:00,0,ACIDENTE
1518,80059,80059,IVINHEMA,MS,0,2014-04-27,11:30:00,0,INCIDENTE
1529,51347,51347,DIADEMA,SP,0,2014-05-04,16:00:00,0,GRAVE
1729,52340,52340,FAMA,MG,0,2014-09-20,15:00:00,1,ACIDENTE
1840,52832,52832,BURITAMA,SP,0,2014-12-14,23:40:00,0,GRAVE
1871,52979,52979,IVINHEMA,MS,0,2015-01-02,14:50:00,0,ACIDENTE
1914,52984,52984,NOVA LIMA,MG,SJLY,2015-02-01,15:10:00,0,ACIDENTE


In [21]:
# filtra ocorrências de cidades que contenham os caracters MA
filtro = df.ocorrencia_cidade.str.contains('MA')
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
12,45396,45396,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0,INCIDENTE GRAVE
16,45693,45693,MACAÉ,RJ,0,2012-01-14,10:30:00,0,INCIDENTE
45,45682,45682,MACAÉ,RJ,0,2012-02-01,13:50:00,0,INCIDENTE
59,45580,45580,MACAÉ,RJ,0,2012-02-07,14:50:00,4,INCIDENTE GRAVE
79,45715,45715,MANAUS,AM,SWFN,2012-02-20,09:10:00,0,INCIDENTE
...,...,...,...,...,...,...,...,...,...
5138,80428,80428,SANTA VITÓRIA DO PALMAR,RS,0,2021-12-11,19:40:00,0,ACIDENTE
5153,80461,80461,MARINGÁ,PR,SBMG,2021-12-24,03:30:00,0,INCIDENTE
5156,80463,80463,MARINGÁ,PR,SBMG,2021-12-28,16:11:00,0,INCIDENTE
5160,80451,80451,MANAUS,AM,SBEG,2021-12-30,14:41:00,0,INCIDENTE


In [22]:
# filtra ocorrências de cidades que contenham os caracters MA ou AL
#filtro = df.ocorrencia_cidade.isin(['MA','AL']')
filtro = df.ocorrencia_cidade.str.contains('MA|AL')

df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
0,52242,52242,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,INCIDENTE
8,45391,45391,CONCEIÇÃO DAS ALAGOAS,MG,0,2012-01-08,16:00:00,0,ACIDENTE
11,52245,52245,PORTO ALEGRE,RS,SBPA,2012-01-09,13:36:00,0,INCIDENTE
12,45396,45396,MARABÁ,PA,SBMA,2012-01-11,11:21:00,0,INCIDENTE GRAVE
16,45693,45693,MACAÉ,RJ,0,2012-01-14,10:30:00,0,INCIDENTE
...,...,...,...,...,...,...,...,...,...
5153,80461,80461,MARINGÁ,PR,SBMG,2021-12-24,03:30:00,0,INCIDENTE
5154,80468,80468,SALVADOR,BA,SBSV,2021-12-26,16:56:00,0,INCIDENTE
5156,80463,80463,MARINGÁ,PR,SBMG,2021-12-28,16:11:00,0,INCIDENTE
5160,80451,80451,MANAUS,AM,SBEG,2021-12-30,14:41:00,0,INCIDENTE


In [23]:
# Outra forma de filtrar por data. A data precisa estar do tipo DateTime
filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
1870,52992,52992,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3,INCIDENTE GRAVE
1871,52979,52979,IVINHEMA,MS,0,2015-01-02,14:50:00,0,ACIDENTE
1872,53073,53073,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0,INCIDENTE
1873,53074,53074,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0,INCIDENTE
1874,52976,52976,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6,ACIDENTE
...,...,...,...,...,...,...,...,...,...
2336,60632,60632,ITABERÁ,SP,0,2015-12-24,14:00:00,0,GRAVE
2337,60600,60600,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0,GRAVE
2338,60642,60642,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0,INCIDENTE
2339,60631,60631,MAÇAMBARÁ,RS,0,2015-12-28,19:00:00,2,ACIDENTE


In [24]:
#  Filtro por Mês e Ano. A data precisa estar do tipo DateTime
filtro = ((df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 10))
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
2231,53492,53492,BELÉM,PA,SBBE,2015-10-01,18:30:00,0,INCIDENTE
2232,53509,53509,CAMPINAS,SP,SBKP,2015-10-01,21:25:00,0,GRAVE
2233,53510,53510,RIBEIRÃO PRETO,SP,SBRP,2015-10-01,19:30:00,0,GRAVE
2234,53513,53513,RIO DE JANEIRO,RJ,SBRJ,2015-10-01,01:48:00,0,INCIDENTE
2235,53511,53511,MANAUS,AM,SWFN,2015-10-03,16:02:00,0,INCIDENTE
2236,53512,53512,BRASÍLIA,DF,SBBR,2015-10-03,14:10:00,0,INCIDENTE
2237,53493,53493,BRASÍLIA,DF,SBBR,2015-10-05,14:15:00,0,INCIDENTE
2238,53497,53497,CORUMBÁ,MS,0,2015-10-06,12:00:00,0,ACIDENTE
2239,53501,53501,CATAS ALTAS,MG,0,2015-10-06,14:33:00,0,ACIDENTE
2240,53504,53504,GOIÂNIA,GO,SBGO,2015-10-07,10:54:00,0,INCIDENTE


In [25]:
#  Filtro por dias, Mês e Ano. A data precisa estar do tipo DateTime
filtro01 = ((df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 10))
filtro02 = ((df.ocorrencia_dia.dt.day > 2) & (df.ocorrencia_dia.dt.day <= 15))
df.loc[filtro01 & filtro02]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao
2235,53511,53511,MANAUS,AM,SWFN,2015-10-03,16:02:00,0,INCIDENTE
2236,53512,53512,BRASÍLIA,DF,SBBR,2015-10-03,14:10:00,0,INCIDENTE
2237,53493,53493,BRASÍLIA,DF,SBBR,2015-10-05,14:15:00,0,INCIDENTE
2238,53497,53497,CORUMBÁ,MS,0,2015-10-06,12:00:00,0,ACIDENTE
2239,53501,53501,CATAS ALTAS,MG,0,2015-10-06,14:33:00,0,ACIDENTE
2240,53504,53504,GOIÂNIA,GO,SBGO,2015-10-07,10:54:00,0,INCIDENTE
2241,53505,53505,GOIÂNIA,GO,SBGO,2015-10-07,11:56:00,0,INCIDENTE
2242,53499,53499,RIO DE JANEIRO,RJ,SBJR,2015-10-08,16:22:00,0,INCIDENTE GRAVE
2243,53502,53502,BRASÍLIA,DF,SBBR,2015-10-08,11:20:00,0,INCIDENTE
2244,53516,53516,FEIRA DE SANTANA,BA,SNJD,2015-10-08,18:05:00,0,INCIDENTE


In [26]:
# Criando nova coluna com data e hora
df['ocorrencia_data_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str)+' '+df.ocorrencia_hora.astype(str))
df['ocorrencia_data_hora']

0      2012-01-05 20:27:00
1      2012-01-06 13:44:00
2      2012-01-06 13:00:00
3      2012-01-06 17:00:00
4      2012-01-06 16:30:00
               ...        
5162   2021-12-30 20:30:00
5163   2021-12-31 09:30:00
5164   2021-12-31 11:59:00
5165   2021-12-31 15:12:00
5166   2021-12-31 20:30:00
Name: ocorrencia_data_hora, Length: 5167, dtype: datetime64[ns]

In [27]:
# Aplicando filtro por data e hora com a nova coluna
filtro01 = df.ocorrencia_data_hora >= '2015-12-03 11:00:00'
filtro02 = df.ocorrencia_data_hora <= '2015-12-08 14:30:00'
df.loc[filtro01 & filtro02]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,classificacao,ocorrencia_data_hora
2307,60637,60637,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,INCIDENTE,2015-12-03 16:47:00
2308,53625,53625,TRINDADE,GO,0,2015-12-06,13:10:00,3,ACIDENTE,2015-12-06 13:10:00
2309,53626,53626,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,GRAVE,2015-12-06 15:00:00
2310,53628,53628,AGUAÍ,SP,0,2015-12-08,14:30:00,1,GRAVE,2015-12-08 14:30:00
2311,53629,53629,JALES,SP,SDJL,2015-12-08,10:20:00,0,GRAVE,2015-12-08 10:20:00
2313,60636,60636,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,INCIDENTE,2015-12-08 13:00:00


In [38]:
# Agrupamento de dados. Agrupando por classificação de incidentes e contando ocorrencias de ocorrencia_uf 
# O método count() ignora valores nulos

df_agrupado = df.groupby(['classificacao']).ocorrencia_uf.count()
df_agrupado

classificacao
ACIDENTE           1317
GRAVE              1261
INCIDENTE          2028
INCIDENTE GRAVE     559
Name: ocorrencia_uf, dtype: int64

In [39]:
# Agrupamento de dados. Agrupando por classificação de incidentes e contando ocorrencias de ocorrencia_uf 
# O método size() não ignora valores nulos

df_agrupado = df.groupby(['classificacao']).ocorrencia_uf.size()
df_agrupado

classificacao
ACIDENTE           1319
GRAVE              1261
INCIDENTE          2028
INCIDENTE GRAVE     559
Name: ocorrencia_uf, dtype: int64

In [40]:
# Agrupamento de dados. Agrupando por classificação de incidentes e contando ocorrencias de ocorrencia_uf 
# O método size() não ignora valores nulos

df_agrupado = df.groupby(['classificacao']).ocorrencia_uf.size().sort_values() # sort_values() ordena por valores
df_agrupado

classificacao
INCIDENTE GRAVE     559
GRAVE              1261
ACIDENTE           1319
INCIDENTE          2028
Name: ocorrencia_uf, dtype: int64

In [42]:
# Agrupamento de dados. Agrupando por classificação de incidentes e contando ocorrencias de ocorrencia_uf 
# O método size() não ignora valores nulos

df_agrupado = df.groupby(['classificacao']).ocorrencia_uf.size().sort_values(ascending=False) # sort_values() ordena por valores
df_agrupado

classificacao
INCIDENTE          2028
ACIDENTE           1319
GRAVE              1261
INCIDENTE GRAVE     559
Name: ocorrencia_uf, dtype: int64

In [43]:
## Total de recomendações que a cidade de Fortaleza teve
filtro = df.ocorrencia_cidade == 'FORTALEZA'
df.loc[filtro].total_recomendacoes.sum()

4

In [46]:
# total de recomendações agrupadas por aerodromo, considera valores NA como aerodromo
df.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum()

ocorrencia_aerodromo
0       627
5JTI      2
9PAA      0
9PBS      0
9PFC      0
       ... 
SWVZ      0
SWXQ      0
SWXV      0
SWYH      0
SWYV      0
Name: total_recomendacoes, Length: 513, dtype: int64

In [48]:
# Filtra apenas cidades que têm recomendações utilizando groupby e loc
filtro = df.total_recomendacoes > 0
df.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_cidade
ACREÚNA        1
AGUAÍ          1
AGUDO          1
ALEGRETE       2
ALMEIRIM       4
              ..
VILHENA        2
VITÓRIA       12
VIÇOSA         1
ÁGUA BOA       3
ÁGUA CLARA     2
Name: total_recomendacoes, Length: 358, dtype: int64

In [49]:
# Filtra cidades que têm recomendações por mês, utilizando groupby e loc
filtro = df.total_recomendacoes > 0
df.loc[filtro].groupby(['ocorrencia_cidade',df.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade  ocorrencia_dia
ACREÚNA            9                  1
AGUAÍ              12                 1
AGUDO              3                  1
ALEGRETE           1                  1
                   10                 1
                                     ..
VITÓRIA            2                 12
VIÇOSA             5                  1
ÁGUA BOA           2                  2
                   12                 1
ÁGUA CLARA         8                  2
Name: total_recomendacoes, Length: 534, dtype: int64